# LangChain Cookbook 👨‍🍳👩‍🍳

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
!pip install accelerate==0.18.0
!pip install transformers==4.28.0


In [67]:
!pip install datasets

In [68]:
from datasets import load_dataset

raw_datasets = load_dataset('code_search_net','python')

In [69]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [70]:
raw_datasets['train'].to_pandas()

,repository_name,func_path_in_repository,func_name,whole_func_string,language,func_code_string,func_code_tokens,func_documentation_string,func_documentation_tokens,split_name,func_code_url
0,adamrehn/ue4cli,ue4cli/Utility.py,Utility.findArgs,"def findArgs(args, prefixes):\n\t\t""""""\n\t\tEx...",python,"def findArgs(args, prefixes):\n\t\t""""""\n\t\tEx...","[def, findArgs, (, args, ,, prefixes, ), :, re...",Extracts the list of arguments that start with...,"[Extracts, the, list, of, arguments, that, sta...",train,https://github.com/adamrehn/ue4cli/blob/f1c345...
1,adamrehn/ue4cli,ue4cli/Utility.py,Utility.stripArgs,"def stripArgs(args, blacklist):\n\t\t""""""\n\t\t...",python,"def stripArgs(args, blacklist):\n\t\t""""""\n\t\t...","[def, stripArgs, (, args, ,, blacklist, ), :, ...",Removes any arguments in the supplied list tha...,"[Removes, any, arguments, in, the, supplied, l...",train,https://github.com/adamrehn/ue4cli/blob/f1c345...
2,adamrehn/ue4cli,ue4cli/Utility.py,Utility.capture,"def capture(command, input=None, cwd=None, she...",python,"def capture(command, input=None, cwd=None, she...","[def, capture, (, command, ,, input, =, None, ...",Executes a child process and captures its output,"[Executes, a, child, process, and, captures, i...",train,https://github.com/adamrehn/ue4cli/blob/f1c345...
3,adamrehn/ue4cli,ue4cli/Utility.py,Utility.run,"def run(command, cwd=None, shell=False, raiseO...",python,"def run(command, cwd=None, shell=False, raiseO...","[def, run, (, command, ,, cwd, =, None, ,, she...",Executes a child process and waits for it to c...,"[Executes, a, child, process, and, waits, for,...",train,https://github.com/adamrehn/ue4cli/blob/f1c345...
4,adamrehn/ue4cli,ue4cli/UnrealManagerBase.py,UnrealManagerBase.setEngineRootOverride,"def setEngineRootOverride(self, rootDir):\n\t\...",python,"def setEngineRootOverride(self, rootDir):\n\t\...","[def, setEngineRootOverride, (, self, ,, rootD...",Sets a user-specified directory as the root en...,"[Sets, a, user, -, specified, directory, as, t...",train,https://github.com/adamrehn/ue4cli/blob/f1c345...
...,...,...,...,...,...,...,...,...,...,...,...
412173,lobocv/anonymoususage,anonymoususage/tools.py,get_rows,"def get_rows(dbconn, tablename, uuid=None):\n ...",python,"def get_rows(dbconn, tablename, uuid=None):\n ...","[def, get_rows, (, dbconn, ,, tablename, ,, uu...",Return all the rows in a table from dbconn\n ...,"[Return, all, the, rows, in, a, table, from, d...",train,https://github.com/lobocv/anonymoususage/blob/...
412174,lobocv/anonymoususage,anonymoususage/tools.py,fetch,"def fetch(dbconn, tablename, n=1, uuid=None, e...",python,"def fetch(dbconn, tablename, n=1, uuid=None, e...","[def, fetch, (, dbconn, ,, tablename, ,, n, =,...",Returns `n` rows from the table's start or end...,"[Returns, n, rows, from, the, table, s, start,...",train,https://github.com/lobocv/anonymoususage/blob/...
412175,lobocv/anonymoususage,anonymoususage/tools.py,get_last_row,"def get_last_row(dbconn, tablename, n=1, uuid=...",python,"def get_last_row(dbconn, tablename, n=1, uuid=...","[def, get_last_row, (, dbconn, ,, tablename, ,...",Returns the last `n` rows in the table,"[Returns, the, last, n, rows, in, the, table]",train,https://github.com/lobocv/anonymoususage/blob/...
412176,lobocv/anonymoususage,anonymoususage/tools.py,get_first_row,"def get_first_row(dbconn, tablename, n=1, uuid...",python,"def get_first_row(dbconn, tablename, n=1, uuid...","[def, get_first_row, (, dbconn, ,, tablename, ...",Returns the first `n` rows in the table,"[Returns, the, first, n, rows, in, the, table]",train,https://github.com/lobocv/anonymoususage/blob/...


In [71]:
print(raw_datasets["train"][123456]["whole_func_string"])

def build_cert_chain(self, flags=SSL_BUILD_CHAIN_FLAG_NONE):
        u'''
        Used for server side only!

        :param flags:
        :return: 1 for success and 0 for failure
        '''
        retVal = SSL_CTX_build_cert_chain(self._ctx, flags)
        return retVal


In [72]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

In [73]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


In [74]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )

training_corpus = get_training_corpus()

In [75]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [76]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

In [ ]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

In [ ]:
tokenizer.save_pretrained("code-search-net-tokenizer")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/test/backup/code-search-net-tokenizer")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
print(type(encoding))

In [ ]:
encoding

In [ ]:
tokenizer.is_fast

In [ ]:
encoding.is_fast

In [ ]:
encoding.tokens()

In [ ]:
encoding.word_ids()

In [ ]:
start, end = encoding.word_to_chars(3)
example[start:end]

In [ ]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
from transformers import pipeline

token_classifier = pipeline("token-classification", aggregation_strategy="simple")
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
outputs


In [ ]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

In [ ]:
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(probabilities)
print(predictions)

In [ ]:
model.config.id2label

In [ ]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )

print(results)

In [ ]:
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

In [ ]:
example[12:14]

In [ ]:
results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != 'O':
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

In [ ]:
example[33:45]

In [ ]:
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # Remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]

        # Grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # The score is the mean of all the scores of the tokens in that grouped entity
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

In [ ]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question_answerer(question=question, context=long_context)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

In [ ]:
import torch

sequence_ids = inputs.sequence_ids()
# 컨텍스트 토큰들을 제외하고는 모두 마스킹한다.
mask = [i != 1 for i in sequence_ids]
# [CLS] 토큰은 마스킹하지 않는다.
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

In [ ]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

In [ ]:
scores = start_probabilities[:, None] * end_probabilities[None, :]

In [ ]:
scores = torch.triu(scores)

In [ ]:
max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(scores[start_index, end_index])

In [ ]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [ ]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index]
}
print(result)

In [ ]:
inputs = tokenizer(question, long_context)
print(len(inputs["input_ids"]))

In [ ]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

In [ ]:
sentence = "This sentence is not too long but we are going to split it anyway."
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)
for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

In [ ]:
print(inputs.keys())

In [ ]:
print(inputs["overflow_to_sample_mapping"])

In [ ]:
sentences = [
    "This sentence is not too long but we are going to split it anyway.",
    "This sentence is shorter but will still get split.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

In [ ]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [ ]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

In [ ]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

In [ ]:
sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
# Mask all the [PAD] tokens
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

In [ ]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [ ]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[0]
    end_idx = idx % scores.shape[0]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

In [ ]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start":start_char, "end":end_char, "score":score}
    print(result)